In [ ]:
import sqlalchemy
import pandas as pd
from binance.client  import Client

In [ ]:
client = Client(api_Key,api_Secret)
#From Binance API library

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')
#The real time data is assigned to the variable 'engine'. Real time data is fetched from 'Live Data Script'

In [ ]:
df = pd.read_sql('BTCUSDT',engine)

In [1]:
df.Price.plot()
#This will plot the graph of the asset price 

NameError: name 'df' is not defined

In [ ]:
#TrendFollowing Stratergy
#If the asset rises by 0.1% in last 60 seconds the trade will be taken.Therefore 0.1 Quantity of the asset will be bought.
#The Exit logic will be +0.15% or -0.15% .
#If the asset price moves up 0.15% the trade will be squared off with a profit.
#If the asset price moves down by 0.15% then trade will be Squared off with a loss 

def stratergy(entry, lookback, qty, open_position=False):
        while True:
            df = pd.read_sql('BTCUSDT',engine)
            lookbackperiod = df.iloc[-lookback:]
            cumret = (lookbackperiod.Price.pct_change() +1).cumprod() - 1
            if not open_position:
                if cumret[cumret.last_valid_index()] > entry:
                    order = client.create_order(symbol='BTCUSDT',
                                               side='BUY',
                                                type='MARKET',
                                               quantity=qty)
                    print(order)
                    open_position = True
                    break
            if open_position:
                while True:
                    df = pd.read_sql('BTCUSDT',engine)
                    sincebuy = df.loc[df.Time > pd.to_datetime(order['transactTime'],
                                                              unit='ms')]
                    if len(sincebuy) > 1:
                        sincebuyret = (sincebuy.Price.pet_change() +1).cumprod() -1
                    last_entry = sicnebuy[sincebuyret.last_valid_index()]
                    if last_entry > 0.0015 or last_entry < -0.0015:
                        order = client.create_order(symbol='BTCUSDT',
                                               side='SELL',
                                                type='MARKET',
                                               quantity=qty)
                    print(order)
                    break

In [ ]:
stratergy(0.001,60,0.001)
#stratergy(% raise in the asset,TimePeriod in Seconds,Qunatity)
#In the above case the asset will be bought if it raises by 0.1% in last 60 seconds. The quantity will be 0.1